In [21]:
import numpy as np
import pandas as pd
#1. 
def import_data(filename):
    file = open(filename,"r")
    X = []
    Y = []
    for i in file.readlines():
        temp = i[:-1].split(",")
        for i in range(len(temp)):
            if temp[i] == "?":
                temp[i] = float("nan")
            else:
                temp[i] = float(temp[i])
        X.append(temp[:-1])
        Y.append(temp[-1])
    return X,Y

#2. 
def impute_missing(X):
    length = len(X)
    two_index = False
    if length%2 == 0:
        two_index = True
    median = [0 for i in range(len(X[0]))]
    for i in range(len(X[0])):
        temp = [X[j][i] for j in range(len(X))]
        temp = [i for i in temp if i==i]
        temp.sort()
        if two_index:
            median[i] = (temp[len(temp)//2] + temp[len(temp)//2-1])/2
        else:
            median[i] = round(temp[len(temp)//2],4)
    for i in range(len(X)):
        for j in range(len(X[i])):
            if X[i][j] != X[i][j]:
                X[i][j] = median[j]
    return X
# Because mean value sometimes is affected by outliers, while a median solves that issue.
def discard_missing(X,Y):
    use_index = [True for i in range(len(X))]
    for i in range(len(X)):
        for j in range(len(X[i])):
            if X[i][j] != X[i][j]:
                use_index[i] = False
    return [X[i] for i in range(len(X)) if use_index[i] == True], [Y[i] for i in range(len(X)) if use_index[i] == True]

def my_shuffle(x,y):
    X = [i for i in range(len(x))]
    new_list = []
    for i in X:
        new_list += [i]
    for i in range(len(X)):
        j = np.random.randint(len(X) - i)
        a = new_list[j]
        new_list[j] = new_list[len(X) - i - 1]
        new_list[len(X) - i - 1] = a
        
    return [x[i] for i in new_list], [y[i] for i in new_list]
      
    
def calculate_std(X):
    means = [0 for i in range(len(X[0]))]
    std = [0 for i in range(len(X[0]))]
    for i in range(len(X[0])):
        count = 0
        temp = [X[j][i] for j in range(len(X))]
        means[i] = sum(temp)/len(temp)
        for j in temp:
            count += (j-means[i])**2
        count = count / (len(temp)-1)
        std[i] = count**(0.5)
    return std, means

def remove_far(X,Y,std,means):
    index = [True for i in range(len(X))]
    for i in range(len(X)):
        for j in range(len(std)):
            if X[i][j] > means[j] + 2 * std[j] or X[i][j] < means[j] - 2 * std[j]:
                index[i] = False
                break
    return [X[i] for i in range(len(index)) if index[i] == True], [Y[i] for i in range(len(index)) if index[i] == True]

def normalize(X, std,means):
    for i in range(len(X)):
        for j in range(len(X[0])):
            if std[j] == 0:
                X[i][j] = 0
            else:
                X[i][j] = (X[i][j] - means[j])/std[j]
    return X

#Time complexity is O(n^2) as it uses two for loops to process each element in the matrix
# and space complexity is O(n^2) also as it stores the whole matrix

#4.
def impute_titanic_data(filepath):
    res = []
    with open(filepath, "r") as f:
        lines = f.readlines()
        for line in lines[1:]:
            line = line[:-1].split(",")
            res.append(line)
    X = []
    Y = []
    for i in range(len(res)):
        res[i][0] = int(res[i][0])
        res[i][1] = int(res[i][1])
        res[i][2] = int(res[i][2])
        if res[i][5] == "male":
            res[i][5] = 1
        if res[i][5] == "female":
            res[i][5] = 0
        if res[i][-1] == "S":
            res[i][-1] = 2
        if res[i][-1] == "C":
            res[i][-1] = 0
        if res[i][-1] == "Q":
            res[i][-1] = 1
        if res[i][6] == "":
            res[i][6] = float("nan")
        else:
            res[i][6] = float(res[i][6])
        if res[i][7] == "":
            res[i][7] = float("nan")
        else:
            res[i][7] = int(res[i][7])
        if res[i][8] == "":
            res[i][8] = float("nan")
        else:
            res[i][8] = int(res[i][8])
        res[i][10] = float(res[i][10])
        Y.append(res[i][1])
        X.append([res[i][0],res[i][2],res[i][5],res[i][6],res[i][7],res[i][8],res[i][10],res[i][-1]])
    return X,Y

#5.
def train_test_split( X, y, t_f ):
    if t_f > 1:
        print("Invalid t_f")
        return 
    length = len(X)
    test_size = length * t_f
    new_x, new_y = my_shuffle(X,y)
    x_train = [new_x[i] for i in range(test_size,length)]
    x_test = [new_x[i] for i in range(test_size)]
    y_train = [new_y[i] for i in range(test_size,length)]
    y_test = [new_y[i] for i in range(test_size)]
    
    return x_train, y_train, x_test, y_test
def train_test_CV_split(X,y,t_f,cv_f):
    if t_f + cs_f > 1:
        print("cannot split")
        return 
    length = len(X)
    test_size = length * t_f
    cv_size = length * cv_f
    new_x, new_y = my_shuffle(X,y)
    x_train = [new_x[i] for i in range(test_size+cv_size,length)]
    x_test = [new_x[i] for i in range(test_size)]
    x_cv = [new_x[i] for i in range(test_size, test_size+cv_size)]
    y_train = [new_y[i] for i in range(test_size+cv_size,length)]
    y_test = [new_y[i] for i in range(test_size)]
    y_cv = [new_y[i] for i in range(test_size,test_size+cv_size)]
    return x_train, y_train, x_test, y_test, x_cv, y_cv
